In [7]:
import cv2 
video_path = r"C:\Users\HP\Downloads\op.webm"  
cap = cv2.VideoCapture(video_path) 
if not cap.isOpened(): 
    print(f"Error: Could not open video file at {video_path}") 
    print("Please verify:") 
    print("1. The file exists at this path") 
    print("2. The file is not corrupted") 
    print("3. The file format is supported by OpenCV") 
    exit() 
fgbg = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=400, detectShadows=True) 
print("Video processing started. Press ESC to exit...") 
while True: 
    ret, frame = cap.read() 
    if not ret: 
        print("End of video or read error") 
        break 
    frame = cv2.resize(frame, (500, 500)) 
    fgmask = fgbg.apply(frame) 
    cv2.imshow('Original Video', frame) 
    cv2.imshow('Foreground Mask', fgmask) 
    if cv2.waitKey(30) & 0xFF == 27: 
        break 
cap.release() 
cv2.destroyAllWindows() 
print("Processing complete")

Video processing started. Press ESC to exit...
Processing complete


In [1]:
import cv2 
import numpy as np 
video_path = r"C:\Users\HP\Downloads\op.webm"  
cap = cv2.VideoCapture(video_path) 
if not cap.isOpened(): 
    print(f"Error: Could not open video file at {video_path}") 
    print("Please check:") 
    print("1. File exists at this path") 
    print("2. File is not corrupted") 
    print("3. File format is supported (try .mp4, .avi)") 
    exit() 
 
background = None 
learning_rate = 0.01  # How quickly background model updates (0-1) 
threshold_value = 25  # Sensitivity to changes (0-255) 
print("Processing video... Press 'q' to quit") 
while True: 
    ret, frame = cap.read() 
    if not ret: 
        print("End of video or read error") 
        break 
    resized_frame = cv2.resize(frame, (500, 500)) 
    gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY) 
    if background is None: 
        background = gray_frame.astype("float") 
        continue 
    cv2.accumulateWeighted(gray_frame, background, learning_rate) 
    background_scaled = cv2.convertScaleAbs(background) 
    diff_frame = cv2.absdiff(gray_frame, background_scaled) 
    _, fg_mask = cv2.threshold(diff_frame, threshold_value, 255, cv2.THRESH_BINARY) 
    fg_mask = cv2.erode(fg_mask, None, iterations=1) 
    fg_mask = cv2.dilate(fg_mask, None, iterations=2) 
    cv2.imshow('Original Video', resized_frame) 
    cv2.imshow('Foreground Mask', fg_mask) 
    if cv2.waitKey(30) & 0xFF == ord('q'): 
        break 
cap.release() 
cv2.destroyAllWindows() 
print("Processing complete") 

Processing video... Press 'q' to quit
End of video or read error
Processing complete


In [2]:
import cv2 
import numpy as np 
video_path = r"C:\Users\HP\Downloads\op.webm"  
cap = cv2.VideoCapture(video_path) 
if not cap.isOpened(): 
    print(f"Error: Could not open video file at {video_path}") 
    print("Please check:") 
    print("1. File exists at this path") 
    print("2. File is not corrupted") 
    print("3. File format is supported") 
    exit() 
bg_subtractor = cv2.createBackgroundSubtractorMOG2( 
    history=500,  
    varThreshold=100, 
    detectShadows=False 
) 
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)) 
scene_change_threshold = 5000000   
frame_diff_threshold = 50 
resize_dim = (500, 500)   
ret, prev_frame = cap.read() 
if not ret: 
    print("Error reading first frame") 
    cap.release() 
    exit() 
prev_frame = cv2.resize(prev_frame, resize_dim) 
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) 
print("Processing video... Press ESC to exit") 
while True: 
    ret, frame = cap.read() 
    if not ret: 
        break 
    frame_resized = cv2.resize(frame, resize_dim) 
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY) 
    diff = cv2.absdiff(prev_gray, gray) 
    _, diff_thresh = cv2.threshold(diff, frame_diff_threshold, 255, cv2.THRESH_BINARY)  
    if np.sum(diff_thresh) > scene_change_threshold: 
        print("Scene change detected - resetting background model") 
        bg_subtractor = cv2.createBackgroundSubtractorMOG2( 
            history=500,  
            varThreshold=100, 
            detectShadows=False 
        ) 
     
    prev_gray = gray.copy() 
    fg_mask = bg_subtractor.apply(frame_resized) 
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)  # Remove noise 
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)  # Fill holes 
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    for contour in contours: 
        if cv2.contourArea(contour) > 100:  # Minimum contour area 
            x, y, w, h = cv2.boundingRect(contour) 
            cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (0, 255, 0), 2) 
    cv2.imshow('Original Video', frame_resized) 
    cv2.imshow('Motion Detection', fg_mask) 
    if cv2.waitKey(30) & 0xFF == 27: 
        break 
cap.release() 
cv2.destroyAllWindows() 
print("Processing complete")

Processing video... Press ESC to exit
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background model
Scene change detected - resetting background

In [3]:
import cv2 
import numpy as np 
def motion_detection(video_path): 
    # Initialize video capture 
    cap = cv2.VideoCapture(video_path) 
    if not cap.isOpened(): 
        print(f"Error: Could not open video file at {video_path}") 
        print("Please verify:") 
        print("1. The file exists at this path") 
        print("2. The file is not corrupted") 
        print("3. You have read permissions") 
        return 
    resize_width, resize_height = 800, 450  # HD resolution (16:9 aspect ratio) 
    learning_rate = 0.01  # Background model update rate 
    threshold_value = 25  # Sensitivity for motion detection 
    scene_change_threshold = 0.15  # 15% of pixels changed = scene change 
    min_contour_area = 500  # Minimum area for detected motion 
    background = None 
    frame_count = 0 
     
    print("Processing video... Press:") 
    print("- 'q' to quit") 
    print("- 'p' to pause") 
    print("- 'r' to reset background model") 
     
    paused = False 
     
    while True: 
        if not paused: 
            ret, frame = cap.read() 
            if not ret: 
                print("End of video reached") 
                break             
            frame_count += 1 
            frame = cv2.resize(frame, (resize_width, resize_height)) 
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
            gray = cv2.GaussianBlur(gray, (5, 5), 0) 
            if background is None: 
                background = gray.astype("float") 
                continue 
            cv2.accumulateWeighted(gray, background, learning_rate) 
            background_scaled = cv2.convertScaleAbs(background) 
            diff = cv2.absdiff(gray, background_scaled) 
            _, thresh = cv2.threshold(diff, threshold_value, 255, cv2.THRESH_BINARY) 
            changed_pixels = cv2.countNonZero(thresh) 
            total_pixels = resize_width * resize_height 
            change_percentage = changed_pixels / total_pixels 
            if change_percentage > scene_change_threshold: 
                print(f"Scene change detected at frame {frame_count} ({change_percentage:.1%} changed)") 
                background = gray.astype("float")  # Reset background model 
                continue 
            thresh = cv2.erode(thresh, None, iterations=1) 
            thresh = cv2.dilate(thresh, None, iterations=2) 
            contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, 
cv2.CHAIN_APPROX_SIMPLE) 
            motion_detected = False 
            for contour in contours: 
                if cv2.contourArea(contour) > min_contour_area: 
                    motion_detected = True 
                    (x, y, w, h) = cv2.boundingRect(contour) 
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2) 
            status = "Motion Detected" if motion_detected else "Monitoring" 
            cv2.putText(frame, f"Status: {status}", (10, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2) 
            cv2.putText(frame, f"Frame: {frame_count}", (10, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2) 
            cv2.imshow("Video Feed", frame) 
            cv2.imshow("Motion Mask", thresh) 
        key = cv2.waitKey(30) & 0xFF 
        if key == ord('q'):  # Quit 
            break 
        elif key == ord('p'):  # Pause 
            paused = not paused 
            print("Video paused" if paused else "Video resumed") 
        elif key == ord('r'):  # Reset background 
            background = None 
            print("Background model reset") 
    cap.release() 
    cv2.destroyAllWindows() 
    print("Processing complete") 
if __name__ == "__main__": 
    video_file = r"C:\Users\HP\Downloads\op.webm"  
    motion_detection(video_file)

Processing video... Press:
- 'q' to quit
- 'p' to pause
- 'r' to reset background model
Scene change detected at frame 35 (28.5% changed)
Scene change detected at frame 38 (17.1% changed)
Scene change detected at frame 41 (20.7% changed)
Scene change detected at frame 45 (43.5% changed)
Scene change detected at frame 48 (31.8% changed)
Scene change detected at frame 49 (16.1% changed)
Scene change detected at frame 50 (20.4% changed)
Scene change detected at frame 51 (20.0% changed)
Scene change detected at frame 54 (18.0% changed)
Scene change detected at frame 58 (17.5% changed)
Scene change detected at frame 65 (16.3% changed)
Scene change detected at frame 66 (15.1% changed)
Scene change detected at frame 70 (20.2% changed)
Scene change detected at frame 71 (16.1% changed)
Scene change detected at frame 73 (19.4% changed)
Scene change detected at frame 74 (19.2% changed)
Scene change detected at frame 75 (23.0% changed)
Scene change detected at frame 76 (19.5% changed)
Scene change